# Hyper Cruises - ELC

In [5]:
import pandas as pd
pd.set_option('display.max_columns', None)
from google.auth import load_credentials_from_file
from google.cloud.bigquery import Client

## Loading data from BigQuery

In [6]:
credentials, project_id = load_credentials_from_file('key.json')

In [7]:
# Load data from BigQuery
client = Client(
    project = project_id,
    credentials=credentials
)

In [8]:
query = "SELECT * FROM `da26-python.music_data.tracks`"

In [9]:
load_job = client.query(query)

In [10]:
data = load_job.to_dataframe()

/Users/antonnyberg/da26-python/da26-python/da26-python/.venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [11]:
def load_data(table):
    query = f"SELECT * FROM `da26-python.music_data.{table}`"
    load_job = client.query(query)
    data = load_job.to_dataframe()
    return data

In [12]:
artists = load_data('artists')

In [13]:
audio_features = load_data('audio_features')

In [14]:
chart_positions = load_data('chart_positions')

In [15]:
mapping = load_data('tracks_artists_mapping')

In [16]:
tracks = load_data('tracks')

## Joining and cleaning

### Joining together tracks, artists, chart_positions and mapping table together

Will later on be joined with the audio_features once cleaned

In [17]:
data = tracks.merge(mapping, on = 'track_id' )

In [18]:
data = data.merge(artists, on = 'artist_id')

In [19]:
data.rename(columns = {'name_x':'track_name', 'name_y':'artist'}, inplace = True)

In [20]:
data = data.merge(chart_positions, on = 'track_id')

In [21]:
data = data[['track_name', 'artist', 'duration_ms', 
             'release_date', 'popularity', 'followers',
             'chart_week', 'list_position', 'track_id', 'artist_id']]

### Cleaning audio_features

Cleaned seperately for better visiblility of columns

- Got rid of null values
- Rounded and changed datatype of tempo-column from float to int
- Dropped redundant time_signature column

In [23]:
# Getting rid of null values
audio_features = audio_features.dropna()

In [24]:
# rounding and changing datatype of tempo to int
audio_features['tempo'] = round(audio_features['tempo']).astype('int')

In [25]:
# dropping redundant columns
audio_features.drop(columns = 'time_signature', inplace=True)

### Cleaning previously joined together data

#### Track duration format from miliseconds to minutes

In [26]:
# Changing track duration format from miliseconds to minutes
data['duration_ms'] = round((data['duration_ms']/1000)/60,1)

In [27]:
data = data.rename(columns = {'duration_ms':'duration_min'})

#### release_date to release_year

In [28]:
data['release_date'] = data['release_date'].str[:4]

In [29]:
data['release_date'] = pd.to_datetime(data['release_date'], format = '%Y')

In [30]:
data['release_date'] = data['release_date'].dt.year

In [31]:
data = data.rename(columns = {'release_date': 'release_year'})

In [32]:
data['chart_week'] = pd.to_datetime(data['chart_week'], format='%Y-%m-%d')

### Joining together data with audio_features

Removing duplicates and resetting index. Filtering for songs released in 2000-2009

In [33]:
data = data.merge(audio_features, on = 'track_id')

In [34]:
data = data.drop_duplicates()

In [35]:
data = data[~data['track_name'].str.contains("christmas")]

In [36]:
data = data.reset_index(drop=True)

In [37]:
cleaned_data = data[(data['release_year'] >= 2000) & (data['release_year'] <= 2009)]

In [39]:
cleaned_data.to_csv('cleaned_data.csv')